In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# table customers
customers_df = pd.read_csv("https://raw.githubusercontent.com/dicodingacademy/dicoding_dataset/main/DicodingCollection/customers.csv")
# customers_df.head()
# table orders
orders_df = pd.read_csv("https://raw.githubusercontent.com/dicodingacademy/dicoding_dataset/main/DicodingCollection/orders.csv")
# orders_df.head()
# table products
product_df = pd.read_csv("https://raw.githubusercontent.com/dicodingacademy/dicoding_dataset/main/DicodingCollection/products.csv")
# product_df.head()
#table sales
sales_df = pd.read_csv("https://raw.githubusercontent.com/dicodingacademy/dicoding_dataset/main/DicodingCollection/sales.csv")
# sales_df.head()

## Assessing Data
# customers_df.info()
customers_df.isna().sum()
# print("Jumlah duplikasi: ", customers_df.duplicated().sum())
# customers_df.describe()
# orders_df.info()
# print("Jumlah duplikasi: ",orders_df.duplicated().sum())
# orders_df.describe()
# product_df.info()
# print("Jumlah duplikasi: ", product_df.duplicated().sum())
# product_df.describe()
# sales_df.info()
sales_df.isna().sum()
# print("Jumlah duplikasi: ", sales_df.duplicated().sum())
# sales_df.describe()

##cleaning data
customers_df.drop_duplicates(inplace=True)
# print("Jumlah duplikasi customer: ", customers_df.duplicated().sum())
customers_df[customers_df.gender.isna()]
customers_df.gender.value_counts()
customers_df.fillna(value="Prefer not to say", inplace=True)
customers_df.isna().sum()
customers_df[customers_df.age == customers_df.age.max()]
customers_df.replace({'age': {customers_df.age.max(): 70}}, inplace=True)
customers_df[customers_df.age == customers_df.age.max()]
customers_df.replace({'age': {customers_df.age.max(): 50}}, inplace=True)
# customers_df.describe()

### cleaning data orders
datetime_columns = ["order_date", "delivery_date"]
 
for column in datetime_columns:
  orders_df[column] = pd.to_datetime(orders_df[column])

# orders_df.info()

###cleaning data product
product_df.drop_duplicates(inplace=True)
# print("Jumlah duplikasi: ", product_df.duplicated().sum())

### cleaning data sales
sales_df[sales_df.total_price.isna()]
sales_df["total_price"] = sales_df["price_per_unit"] * sales_df["quantity"]
sales_df.isna().sum()

### explorasi data customer_df
customers_df.describe(include="all")

### demografi berdasarkan jenis kelamin
customers_df.groupby(by="gender").agg({
  "customer_id": "nunique",
  "age": ["max", "min", "mean", "std"]
})

### persebaran pelanggan berdasarkan city dan state
customers_df.groupby(by="city").customer_id.nunique().sort_values(ascending=False)
customers_df.groupby(by="state").customer_id.nunique().sort_values(ascending=False)

### explorasi data orders_df memubuat sebuah kolom baru untuk memuat setiap order
delivery_time = orders_df["delivery_date"] - orders_df["order_date"]
delivery_time = delivery_time.apply(lambda x: x.total_seconds())
orders_df["delivery_time"] = round(delivery_time/86400)
print(orders_df["delivery_time"])
orders_df.describe(include="all")

### explorasi data orders_df dan customer_df
customer_id_in_orders_df =  orders_df.customer_id.tolist()
customers_df["status"] = customers_df["customer_id"].apply(lambda x: "Active" if x in customer_id_in_orders_df else "Non Active")
customers_df.sample(5)
customers_df.groupby(by="status").customer_id.count()

orders_customers_df = pd.merge(
    left=orders_df,
    right=customers_df,
    how="left",
    left_on="customer_id",
    right_on="customer_id"
)
orders_customers_df.head()

### jumlah order berdasarkan kota
orders_customers_df.groupby(by="city").order_id.nunique().sort_values(ascending=False).reset_index().head(10)

### jumlah order berdasarkan state
orders_customers_df.groupby(by="state").order_id.nunique().sort_values(ascending=False)

### jumlah order berdasarkan gender
orders_customers_df.groupby(by="gender").order_id.nunique().sort_values(ascending=False)

### jumlah order berdasarkan kelompok usia
orders_customers_df["age_group"] = orders_customers_df.age.apply(lambda x: "Youth" if x <= 24 else ("Seniors" if x > 64 else "Adults"))
orders_customers_df.groupby(by="age_group").order_id.nunique().sort_values(ascending=False)


# Explorasi data product_df dan sales_df
product_df.describe(include="all")
sales_df.describe(include="all")

### produk harga tertinggi dan terendah
product_df.sort_values(by="price", ascending=False)

product_df.groupby(by="product_type").agg({
    "product_id": "nunique",
    "quantity": "sum",
    "price":  ["min", "max"]
})
 
product_df.groupby(by="product_name").agg({
    "product_id": "nunique",
    "quantity": "sum",
    "price": ["min", "max"]
})

### produk terlaris
sales_product_df = pd.merge(
    left=sales_df,
    right=product_df,
    how="left",
    left_on="product_id",
    right_on="product_id"
)
sales_product_df.head()

### informasi penjualan produk berdasarkan product_type
sales_product_df.groupby(by="product_type").agg({
    "sales_id": "nunique",
    "quantity_x": "sum",
    "total_price": "sum"
})

### informasi penjualan produk berdasarkan product_name
sales_product_df.groupby(by="product_name").agg({
    "sales_id": "nunique",
    "quantity_x": "sum",
    "total_price": "sum"
}).sort_values(by="total_price", ascending=False)

# Explorasi data all
all_df = pd.merge(
    left=sales_product_df,
    right=orders_customers_df,
    how="left",
    left_on="order_id",
    right_on="order_id"
)
all_df.head()

### preferensi pembelian berdasarkan state pelanggan
all_df.groupby(by=["state", "product_type"]).agg({
    "quantity_x": "sum",
    "total_price": "sum"
})

### selera tipe produk pelanggan berdasarkan gender dan kelompok usia
all_df.groupby(by=["gender", "product_type"]).agg({
    "quantity_x": "sum",
    "total_price": "sum"
})

all_df.groupby(by=["age_group", "product_type"]).agg({
    "quantity_x": "sum",
    "total_price": "sum"
})




0      25.0
1      10.0
2      26.0
3      13.0
4       1.0
       ... 
995    20.0
996    13.0
997    16.0
998    27.0
999    17.0
Name: delivery_time, Length: 1000, dtype: float64


quantity_x  total_price
age_group product_type                         
Adults    Jacket              2292       245055
          Shirt               2225       227781
          Trousers            2272       231271
Seniors   Jacket               777        82959
          Shirt                733        74977
          Trousers             766        77471
Youth     Jacket               274        29012
          Shirt                301        30842
          Trousers             322        32432